In [1]:
import os

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy
from pytorch_lightning.callbacks import ModelCheckpoint

import os
import tqdm
import copy
import random
import logging
from absl import app
from absl import flags
from torch.utils.data import TensorDataset, DataLoader

import nam.metrics
import nam.data
from nam.model import *
from nam.pl import *
from datetime import datetime

from pytorch_lightning.loggers import WandbLogger
import wandb

FLAGS = flags.FLAGS

In [2]:
config = defaults()
print(config)

Config(activation='exu', batch_size=1024, cross_val=False, data_path='data/GALLUP.csv', decay_rate=0.995, device='cpu', dropout=0.5, early_stopping_patience=50, experiment_name='NAM', feature_dropout=0.25, hidden_layer='relu', hidden_units=[64, 32, 32], id_fold=1, l2_regularization=0.001, learning_rate=0.00674, logdir='output', n_basis_functions=1024, n_folds=5, n_splits=3, num_epochs=10, num_models=1, output_regularization=0.01, regression=True, save_model_frequency=2, save_top_k=3, seed=2021, shallow_layer='exu', shuffle=True, units_multiplier=2, use_dnn=False, wandb=False)


In [3]:
wandb.init(project="nam-final", config=config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: amrmkayid (use `wandb login --relogin` to force relogin)


In [4]:
nam_data = NAMDataModule(config)

In [5]:
# init model
nam_model = NAM(config=config, x_train=nam_data.tmp_x_train)
nam_model

NAM(
  (model): NeuralAdditiveModel(
    (feature_nns): ModuleList(
      (0): FeatureNN(
        (layers): ModuleList(
          (0): ExULayer()
          (1): ReLULayer()
          (2): ReLULayer()
          (3): ReLULayer()
        )
        (dropout): Dropout(p=0.5, inplace=False)
        (linear): Linear(in_features=32, out_features=1, bias=False)
      )
      (1): FeatureNN(
        (layers): ModuleList(
          (0): ExULayer()
          (1): ReLULayer()
          (2): ReLULayer()
          (3): ReLULayer()
        )
        (dropout): Dropout(p=0.5, inplace=False)
        (linear): Linear(in_features=32, out_features=1, bias=False)
      )
      (2): FeatureNN(
        (layers): ModuleList(
          (0): ExULayer()
          (1): ReLULayer()
          (2): ReLULayer()
          (3): ReLULayer()
        )
        (dropout): Dropout(p=0.5, inplace=False)
        (linear): Linear(in_features=32, out_features=1, bias=False)
      )
      (3): FeatureNN(
        (layers): ModuleL

In [6]:
wandb_logger = WandbLogger()

# optional: log model topology
wandb_logger.watch(nam_model.model)

In [7]:
# Initialize a trainer
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    dirpath=f"/home/amrmkayid/nam/models/{datetime.now().strftime('%Y%m%d_%H%M')}/",
    filename="nam-{epoch:02d}-{val_loss:.2f}-{MAE_epoch:.2f}",
    save_top_k=3,
    mode="min",
)
trainer = pl.Trainer(max_epochs=config.num_epochs, logger=wandb_logger, callbacks=[checkpoint_callback])  #, progress_bar_refresh_rate=20)

# # Train the model ⚡
trainer.fit(nam_model, nam_data)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
wandb: WARNING Serializing object of type ndarray that is 462456 bytes

  | Name  | Type                | Params
----------------------------------------------
0 | model | NeuralAdditiveModel | 505 K 
----------------------------------------------
505 K     Trainable params
0         Non-trainable params
505 K     Total params
2.022     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/amrmkayid/miniconda3/envs/nam/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:326: UserWarning: The number of training samples (43) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [8]:
trainer.test(datamodule=nam_data)

/home/amrmkayid/miniconda3/envs/nam/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.prepare_data has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.prepare_data.
  rank_zero_deprecation(
wandb: WARNING Serializing object of type ndarray that is 462456 bytes


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_MAE': 0.8534360527992249, 'test_loss': 1.3207871913909912}
--------------------------------------------------------------------------------


[{'test_loss': 1.3207871913909912, 'test_MAE': 0.8534360527992249}]